In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

In [ ]:
data=pd.read_csv(r'../Data/input/Twitter_uncleaned_data.csv')
# Extract posting year
data['time']= pd.to_datetime(data['发布时间'],format='%a %b %d %H:%M:%S +0000 %Y')
data['year'] = data['time'].year

All data clean

In [ ]:
# Removal of duplicate data
data=data.drop_duplicates(subset=['推文链接'],keep='last')
# Lowercase the data
data['text_cleaned'] = data['文字内容'].apply(lambda x: x.lower())
# Filtering data containing related words
keywords = ['urban air mobility', 'uam ','air taxi','evtol','urban aerial mobility','drone taxi','autonomous air vehicle','vtol','flying car','mobility','uav','advenced air','eve','ehang','jaunt','joby','aviation','volocopter']
data = data[data["text_cleaned"].str.contains('|'.join(keywords))]
# Remove useless characters from text
data['text_cleaned'] = data['文字内容'].str.replace('\n', ' ')
data['text_cleaned'] = data['text_cleaned'].apply(lambda x: re.sub(r'@(\w+)', '', x))
data['text_cleaned'] = data['text_cleaned'].apply(lambda x: re.sub(r'#(\w+)', '', x))

Keyword "VTOL" data clean

In [ ]:
data_vtol=data[data['关键词']=='vtol']
keywords = ['enemy','wars',' war','war ','$vtol','army','weapon','battle','military','soldier']
data_vtol=data_vtol[-(data_vtol["text_cleaned"].str.contains('|'.join(keywords)))]

In [ ]:
data=data[-(data['关键词']=='vtol')]
data=pd.concat(data,data_vtol)

Nation identify

In [ ]:
import openai
import pandas as pd
import numpy as np
import os
import ast
import time
import sys
import csv

In [ ]:
# Call openai to recognize the country
openai.api_key = "sk-uZNblLpNRmFyvzStF39f4d9d5353430a8792E1134bB95f24"
openai.base_url = "https://api.gpt.ge/v1/"
openai.default_headers = {"x-foo": "true"}

def askquestion(location):
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Please identify the full name of the country represented by each element of the list (location or latitude/longitude) and return a list of the same length, making sure to follow the format ['xxx','xxx','xxx'] strictly, or answer unkonwn if it's not clear, and also answer unkonwn if there are more than one country contained in a single element"},
            {"role": "user", "content": location},
        ],
    )
    return completion.choices[0].message.content

In [ ]:
# Clean nation text
data['location']=data["博主位置"].copy()
data['location'].fillna('null', inplace=True)
filtered_rows = data['location'].str.match(r'^[0-9. +-]+$')
data.loc[filtered_rows, 'location'] = 'useless'
keywords = ['World', '.com','world','earth','Global','global','Earth']
filtered_rows =data['location'].str.contains('|'.join(keywords), case=False)
data.loc[filtered_rows, 'location'] = 'useless'

In [ ]:
list_all=[]
a=10

for i in range(0, int(len(data)/a)):
    tag=True
    count=0
    while tag and count<=5:
        try:
            dic={}
            dic['No.']=i*a
            # 删除每个元素中的符号
            input_list=data['location'].iloc[i*a:i*a+a].tolist()
            cleaned_list = [address.replace("'", '') for address in input_list]
            input_location = str(cleaned_list)
            dic['input']=input_location
            dic['input_count']=len(input_list)
            nation=askquestion(input_location)
            dic['output']=nation
            output_nation = ast.literal_eval(nation)    # 将文本转换为列表
            dic['output_count']=len(output_nation)
            print(str(i)+'/'+str(len(data)/a))
            fp = open(f"./Data/interim/twitter_nation.csv", 'a+', encoding='utf-8-sig', newline='')
            csv_fp = csv.writer(fp)
            csv_fp.writerow(dic.values())
            fp.close()
            tag=False
        except Exception as e:
            print(str(i)+'出错了')
            tag=True
            count=count+1

# After calling openai for recognition, some recognition errors need to be manually checked and corrected

nation_clean

In [ ]:
data_nation=pd.read_csv(r'../Data/interim/twitter_nation.csv')
merged_list = []
data_nation['output_list'].apply(lambda y:merged_list.extend(ast.literal_eval(y)))
my_series = pd.Series(merged_list)
nation_list = pd.DataFrame()
nation_list['nation'] = my_series
# Add nation to all twitter data
data_final=pd.concat([data,nation_list], axis=1)

In [ ]:
# Lowercase the data
data_final['nation_cleaned'] = data_final['nation'].apply(lambda x: x.lower())
#nation clean
data_final['nation_cleaned'].replace(['undefined','country unknown','erusea',' unknown','africa','greenland','unbekannt','unnonwn','bikini bottom','international waters','open skies','sky','belka','lesotho','east timor','international space station','western sahara','guam','un','aus','uam','eu','udknown','metaverse','known','undef','north'], 'unknown', inplace=True)
data_final['nation_cleaned'].replace(['undefined','unkonwn','international','european union','unkown', 'unnown','europe','mars','unavailable','antarctica','earth','unknonwn','unknwon','asia','unlnown','unknonw','multiple countries','unidentified','world'], 'unknown', inplace=True)
data_final['nation_cleaned'].replace(['united state',' united states','san francisco','united states ','wisconsin','new jersey','united states.','northern mariana islands','us','washington','usa','united sates','united states of america','puerto rico','united states and united kingdom','texas','new hampshire','north carolina','united states virgin islands'], 'united states', inplace=True)
data_final['nation_cleaned'].replace(['montserrat','falkland islands',' united kingdom','england','uk','scotland','isle of man','wales','gibraltar','guernsey','jersey','cayman islands','anguilla','saint helena'], 'united kingdom', inplace=True)
data_final['nation_cleaned'].replace(['hong kong','taiwan','macau','macao','peoples republic of china'], 'china', inplace=True)
data_final['nation_cleaned'].replace(['vatican city','vatican'], 'holy see', inplace=True)
data_final['nation_cleaned'].replace(['the netherlands','nederland','aruba','saint martin','sint maarten','south netherlands','Curaçao','curaçao'], 'netherlands', inplace=True)
data_final['nation_cleaned'].replace(['republic of korea'], 'south korea', inplace=True)
data_final['nation_cleaned'].replace(['kosovo'], 'serbia', inplace=True)
data_final['nation_cleaned'].replace(['new caledonia'], 'france', inplace=True)
data_final['nation_cleaned'].replace(['democratic republic of the congo','republic of the congo'], 'congo', inplace=True)
data_final['nation_cleaned'].replace(['republic of the philippines'], 'philippines', inplace=True)
data_final['nation_cleaned'].replace(['the bahamas'], 'bahamas', inplace=True)
data_final['nation_cleaned'].replace(['brasil'], 'brazil', inplace=True)
data_final['nation_cleaned'].replace(['republic of croatia'], 'croatia', inplace=True)
data_final['nation_cleaned'].replace(['islamic republic of iran'], 'iran', inplace=True)
data_final['nation_cleaned'].replace(['canada and united states','canada '], 'canada', inplace=True)
data_final['nation_cleaned'].replace(['ivory coast'], "côte d'ivoire", inplace=True)
data_final['nation_cleaned'].replace(['kingdom of morocco'], 'morocco', inplace=True)
data_final['nation_cleaned'].replace(['russian federation'], 'russia', inplace=True)
data_final['nation_cleaned'].replace(['lebannon'], 'Lebanon', inplace=True)
data_final['nation_cleaned'].replace(['kingdom of saudi arabia'], 'saudi arabia', inplace=True)
data_final['nation_cleaned'].replace(['federated states of micronesia'], 'micronesia', inplace=True)
data_final['nation_cleaned'].replace(['germany '], 'germany', inplace=True)
data_final['nation_cleaned'].replace(['islamic republic of pakistan'], 'pakistan', inplace=True)

In [ ]:
data_final.to_csv('../Data/interim/twitter_nation_cleaned.csv', encoding="utf_8_sig",index=True)
# After the initial cleaning of the nation text using the code, there were some minor errors that needed to be cleaned manually

Retain useful data for next statistical analysis

In [ ]:
data=pd.read_csv(r'./Data/interim/twitter_nation_cleaned.csv')
data_retained = data[['关键词', '推文链接', '博主ID号','账户验证类型','nation_cleaned','year','text_cleaned']]
useful_data = pd.DataFrame(data_retained)
# Regularization of column names
new_column_names = {'关键词': 'keyword', '推文链接': 'link', '博主ID号': 'user_id', '账户验证类型': 'user_type', 'nation_cleaned': 'nation', 'year': 'year', 'text_cleaned': 'text_cleaned'}
useful_data = useful_data.rename(columns=new_column_names)
useful_data.to_csv('../Data/interim/twitter_text.csv', encoding="utf_8_sig",index=False)

Concat Twitter data and Weibo data

In [ ]:
data_twitter=pd.read_csv(r'./Data/interim/twitter_nation_cleaned.csv')
data_weibo=pd.read_csv(r'./Data/interim/weibo_nation_cleaned.csv')
data_all=pd.concat([data_twitter,data_weibo], axis=1)
data_all.to_csv('../Data/interim/all_data_preprocessed.csv', encoding="utf_8_sig",index=False)